論文<br>
https://arxiv.org/abs/2303.09535<br>
<br>
GitHub<br>
https://github.com/chenyangqiqi/fatezero<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/FateZero_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup environment

## git clone

In [ ]:
%cd /content

!git clone https://github.com/ChenyangQiQi/FateZero

%cd /content/FateZero
# Commits on Mar 22, 2023
!git checkout 37a321f04a41c0c342e78b2dca0f02183d2ba1de

## install libraries

In [ ]:
%cd /content/FateZero

!pip install -r requirements.txt
!pip install -q -U --pre triton
!pip install -q diffusers[torch]==0.11.1 transformers==4.26.0 bitsandbytes==0.35.4 decord accelerate omegaconf einops ftfy gradio xformers
!pip install moviepy yt-dlp
!pip install imageio==2.4.1

## import libraries

In [ ]:
%cd /content/FateZero

import os
import glob

import torch
import cv2

from yt_dlp import YoutubeDL
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

from omegaconf import OmegaConf

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device is', device)

# download pretrain models

In [ ]:
%cd /content/FateZero

MODEL_NAME = "CompVis/stable-diffusion-v1-4"

!git lfs install
!git clone https://huggingface.co/$MODEL_NAME ckpt/$MODEL_NAME
MODEL_NAME = f"./ckpt/{MODEL_NAME}"
print(f"[*] MODEL_NAME={MODEL_NAME}")

# download video

In [ ]:
%cd /content/FateZero

video_url = 'https://www.youtube.com/watch?v=YAWt-gktZqM' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  77#@param {type:"integer"}
end_sec =  78#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
download_resolution = 720
full_video_path = './data/full_video02.mp4'
input_clip_path = './data/clip_video02.mp4'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

In [ ]:
clip = VideoFileClip(input_clip_path)
clip = resize(clip, height=420)
clip.ipython_display()

## video to frames

In [ ]:
# load video
video = cv2.VideoCapture(input_clip_path)
# get video info
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('video width:', w, ', height:', h, ', fps:', frames_per_second, ', frames:', num_frames)

# create out directory
out_path = './data/frames'
os.makedirs(out_path, exist_ok=True)


In [ ]:
# Get only 8frames due to OOM
interval = int(num_frames/8)

for i in range(num_frames):
  if i % interval == 0:
    success, frame = video.read()
    cv2.imwrite( os.path.join(out_path, f'{i:06}.jpg'), cv2.resize(frame, (512, 512)))


# set parameters

In [ ]:
!rm -rf /content/FateZero/result

In [ ]:
#@markdown frame image directory
VIDEO_FILE = './data/frames'
VIDEO_ID = VIDEO_FILE.split('/')[-1] # directory name
RESULT_DIR = 'result/'+VIDEO_ID # output directory
CONFIG_NAME = "config/"+VIDEO_ID+".yaml" # config path

#@markdown source prompt
source_prompt = "a black car driving down a curvy road in the countryside" #@param {type:"string"}
#@markdown edit prompt
edit_prompt = "watercolor painting of a black car driving down a curvy road in the countryside"  #@param {type:"string"}
EMPHYSIS_WORD = "watercolor" #@param {type:"string"}
EMPHYSIS_VALUE = 10 #@param {type:"number"}
INVERSION_STEP = 8 #@param {type:"number"}
REPLACE_STRENGTH = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
STORE_ATTENTION_ON_disk = False #@param {type:"boolean"}

# OOM avoidance
# video_length = num_frames
video_length = 8
# width = w
# height = h
width = 512 
height = 288 

In [ ]:
config = {
  "pretrained_model_path": MODEL_NAME,
  "logdir": RESULT_DIR,
  "train_dataset": {
    "path": VIDEO_FILE,
    "prompt": source_prompt,
    "n_sample_frame": video_length,
    "sampling_rate": 1,
    "stride": 80,
    "offset": 
    {
        "left": 0,
        "right": 0,
        "top": 0,
        "bottom": 0,
    }
  },
  "validation_sample_logger_config":{
      "use_train_latents": True,
      "use_inversion_attention": True,
      "guidance_scale": 7.5,
      "prompts":[
          source_prompt,
          edit_prompt,
      ],
      "p2p_config":[ 
          {
          "cross_replace_steps":{
              "default_":0.8
              },
          "self_replace_steps": 0.8,
          "masked_self_attention": True,
           "bend_th": [2, 2],
          "is_replace_controller": False 
          },
          {
          "cross_replace_steps":{
              "default_":0.8
              },
          "self_replace_steps": 0.8,
          "eq_params":{
              "words":[EMPHYSIS_WORD],
              "values": [EMPHYSIS_VALUE]
            },
          "use_inversion_attention": True,
          "is_replace_controller": False 
          }]
          ,
    "clip_length": "${..train_dataset.n_sample_frame}",
    "sample_seeds": [0],
    "num_inference_steps": INVERSION_STEP,
    "prompt2prompt_edit": True
     },
  "disk_store": STORE_ATTENTION_ON_disk,
  "model_config":{
      "lora": 160,
      "SparseCausalAttention_index": ['mid'],
      "least_sc_channel": 640
  },
  "test_pipeline_config":{
    "target": "video_diffusion.pipelines.p2pDDIMSpatioTemporalPipeline.p2pDDIMSpatioTemporalPipeline",
    "num_inference_steps": "${..validation_sample_logger.num_inference_steps}"
  },
  "epsilon": 1e-5,
  "train_steps": 10,
  "seed": 0,
  "learning_rate": 1e-5,
  "train_temporal_conv": False,
  "guidance_scale": "${validation_sample_logger_config.guidance_scale}"
}

OmegaConf.save(config, CONFIG_NAME)
print('save new configue to ', CONFIG_NAME)

# inference

In [ ]:
%cd /content/FateZero

!python test_fatezero.py --config=$CONFIG_NAME

## before

In [ ]:
clip = VideoFileClip(glob.glob('/content/FateZero/result/*/sample/step_0_0_0.mp4')[0])
clip = resize(clip, height=420)
clip.ipython_display()

## after

In [ ]:
clip = VideoFileClip(glob.glob('/content/FateZero/result/*/sample/step_0_1_0.mp4')[0])
clip = resize(clip, height=420)
clip.ipython_display()